In [1]:
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
import numpy as np
import scipy as sp
import pandas as pd
# import geopandas as gpd
# import geoplot as gplt
from shapely.geometry import Point
import shapely
import reverse_geocoder as rg

import sklearn as sk
import matplotlib as mpl
import matplotlib.pylab as plt
import matplotlib.font_manager as fm
from mpl_toolkits.mplot3d import Axes3D

import seaborn as sns
sns.set(rc={'figure.figsize':(13.7,10.27)})
sns.set_style("whitegrid")
sns.set_color_codes()

In [2]:
from dask.distributed import Client
import dask.bag as db
import dask.dataframe as dd
import dask.array as da
import dask

from ast import literal_eval
from collections.abc import MutableMapping
from collections import Counter, defaultdict
# import h5py
import io
import os

In [3]:
from surprise import BaselineOnly
from surprise import Dataset
from surprise import Reader
from surprise.model_selection.split import train_test_split
from surprise.model_selection import cross_validate, GridSearchCV
import pandas as pd
import numpy as np
import os, io
from surprise import KNNBasic, KNNWithMeans, KNNWithZScore, KNNBaseline
from surprise import SVDpp, SVD, NMF
from surprise import SlopeOne
from surprise import CoClustering
from surprise import accuracy
from sklearn.linear_model import Ridge, RidgeCV, SGDRegressor
from sklearn.metrics import mean_squared_error as mse
import math

In [4]:
SVD_ALGO = "SVD"
BLENDER_RIDGE = "RidgeCV"
BLENDER_SGD = "SGDRegressor"

In [5]:
client = Client(n_workers=8)
client

Client Scheduler: tcp://127.0.0.1:44781 Dashboard: http://127.0.0.1:8787/status,Cluster Workers: 8 Cores: 16 Memory: 33.65 GB


In [6]:
joined_df = dd.read_csv('data/joined_df.csv').compute()
joined_df = joined_df.loc[:, ~joined_df.columns.str.match('Unnamed')]
joined_df.columns

Index(['rating', 'reviewerName', 'categories', 'gPlusPlaceId', 'gPlusUserId',
       'user_lat', 'user_long', 'placeName', 'price', 'address', 'place_lat',
       'place_long'],
      dtype='object')

In [7]:
def places_filter(df, lat_l, lat_h, lon_l, lon_h):
    return df[(lat_l <= df.place_lat ) & (df.place_lat <= lat_h) & (lon_l <= df.place_long) & (df.place_long <= lon_h)]

In [8]:
%%time
usa_df = places_filter(joined_df, 19.50139, 64.85694, -161.75583, -68.01197)
usa_df.shape

CPU times: user 208 ms, sys: 40 ms, total: 248 ms
Wall time: 239 ms


(2285757, 12)

In [9]:
# Create training set
reader = Reader(rating_scale=(1,5))  #invoke reader instance of surprise library
data = Dataset.load_from_df(usa_df[['gPlusUserId','gPlusPlaceId','rating']], reader)

# Starting Stacking for Collaborative filtering RecSys

# Training first layer for Stacking!

In [10]:
def train_first_layer(algorithms, hparams, train_set, test_set):
    trained_algorithms = []
    performances = []
    
    for idx, algorithm in enumerate(algorithms):
        hparam = hparams[idx]
        print("\t\t{} training started: {}".format(idx+1, hparam))
        if algorithm is SVD or algorithms is SVDpp: configured_algorithm = algorithm(n_factors=hparam["n_factors"], n_epochs=hparam["n_epochs"], reg_all= hparam["reg_all"])
        elif algorithm is NMF: configured_algorithm = algorithm(n_factors=hparam["n_factors"], n_epochs=hparam["n_epochs"])
        elif algorithm is SlopeOne: configured_algorithm = algorithm()
        elif algorithm is CoClustering: configured_algorithm = algorithm(n_cltr_u=hparam["n_cltr_u"], n_cltr_i=hparam["n_cltr_i"], n_epochs=hparam["n_epochs"], random_state=42)
        configured_algorithm.fit(train_set)
        trained_algorithms.append(configured_algorithm)
        rmse = accuracy.rmse(configured_algorithm.test(test_set), verbose=False)
        print("\t\tRMSE= ", rmse)
        performances.append(rmse)
    
    return trained_algorithms, performances

In [11]:
def predict_first_layer(trained_algorithms, test_set):
    return [
        trained_algorithm.test(test_set)
        for trained_algorithm in trained_algorithms
    ]

In [12]:
def train_last_layer(trained_algorithms, train_set, blender_algorithm=BLENDER_RIDGE):
    actual_trainingSet = [each for each in train_set.all_ratings()]
    predictions_for_trainingSet = predict_first_layer(trained_algorithms, actual_trainingSet)
    print("\t\tGenerating predictions Complete !")
    
    train_pred = [[each.est for each in prediction] for prediction in predictions_for_trainingSet]
    train_true = [each.r_ui for each in predictions_for_trainingSet[0]]
    
    blender_train_X = np.column_stack(train_pred)
    blender_train_y = train_true
    print("\t\tOrganizing data for Blender Complete !")
    
    assert blender_train_X.shape[0] == len(blender_train_y), "There's problem in dimension for training set"
    
    blender = None
    if blender_algorithm is BLENDER_RIDGE: blender = RidgeCV(cv=5)
    elif blender_algorithm is BLENDER_SGD: blender = SGDRegressor(max_iter=5000)
    print("\t\tDetermination for Blender Algorithm Complete !")    
    
    blender.fit(blender_train_X, blender_train_y)
    print("\t\tBlender Training Complete !")
    return blender

In [13]:
def predict_last_layer(trained_algorithms, blender, userID, iid):
    preds = [algo.predict(userID, iid) for algo in trained_algorithms]
    blender_X = np.column_stack((pred.est for pred in preds))
    return blender.predict(blender_X)

In [14]:
def CV(cv, algorithms, hparams):
    individual_performances = []
    sgd_performances = []
    ridge_performances = []
    
    for idx in range(cv):
        print("{} Cross Validation started".format(idx+1))
        trainingSet, testSet = train_test_split(data, test_size=0.2, train_size=None, random_state=42, shuffle=True)
        print("\tData Perparation Completed")
        
        trained_algorithms, performances = train_first_layer(algorithms, hparams, trainingSet, testSet)
        individual_performances.append(performances)
        
        predictions = predict_first_layer(trained_algorithms, testSet)
        test_true = [each.r_ui for each in predictions[0]]
        test_pred = [[each.est for each in prediction] for prediction in predictions]
        blender_test_X = np.column_stack(test_pred)
        blender_test_y = test_true
        print("\tPreparing data for Blender Completed")
        
        SGD_blender = train_last_layer(trained_algorithms, trainingSet, BLENDER_SGD)
        final_pred = SGD_blender.predict(blender_test_X)
        sgd_performance = math.sqrt(mse(final_pred, blender_test_y))
        sgd_performances.append(sgd_performance)
        print("\tSGD Performance: ", sgd_performance)
        
        RIDGE_blender = train_last_layer(trained_algorithms, trainingSet, BLENDER_RIDGE)
        final_pred = RIDGE_blender.predict(blender_test_X)
        ridge_performance = math.sqrt(mse(final_pred, blender_test_y))
        ridge_performances.append(ridge_performance)
        print("\tRidge Performance: ", ridge_performance)
    
    return individual_performances, sgd_performances, ridge_performances

In [15]:
# Configuration 1
algorithms = [SVD, SVD, SVD]
hparams = [
    {
        "n_factors": 20,
        "n_epochs": 20,
        "reg_all": 0.02
    },
    {
        "n_factors": 10,
        "n_epochs": 10,
        "reg_all": 0.02
    },
    {
        "n_factors": 40,
        "n_epochs": 40,
        "reg_all": 0.02
    }
]
individual_performances1, sgd_performances1, ridge_performances1 = CV(3, algorithms, hparams)

1 Cross Validation started
	Data Perparation Completed
		1 training started: {'n_factors': 20, 'n_epochs': 20, 'reg_all': 0.02}
		RMSE=  1.104058928221667
		2 training started: {'n_factors': 10, 'n_epochs': 10, 'reg_all': 0.02}
		RMSE=  1.1163021178376735
		3 training started: {'n_factors': 40, 'n_epochs': 40, 'reg_all': 0.02}
		RMSE=  1.1024902835441712
	Preparing data for Blender Completed
		Generating predictions Complete !
		Organizing data for Blender Complete !
		Determination for Blender Algorithm Complete !
		Blender Training Complete !
	SGD Performance:  1.108210460939158
		Generating predictions Complete !
		Organizing data for Blender Complete !
		Determination for Blender Algorithm Complete !
		Blender Training Complete !
	Ridge Performance:  1.1741878518117768
2 Cross Validation started
	Data Perparation Completed
		1 training started: {'n_factors': 20, 'n_epochs': 20, 'reg_all': 0.02}
		RMSE=  1.1041334500931705
		2 training started: {'n_factors': 10, 'n_epochs': 10, 'reg

In [16]:
# Configuration 2
algorithms = [SVD, SVDpp, NMF]
hparams = [
    {
        "n_factors": 20,
        "n_epochs": 20,
        "reg_all": 0.02
    },
    {
        "n_factors": 10,
        "n_epochs": 10,
        "reg_all": 0.02
    },
    {
        "n_factors": 10,
        "n_epochs": 10
    }
]
individual_performances2, sgd_performances2, ridge_performances2 = CV(3, algorithms, hparams)

1 Cross Validation started
	Data Perparation Completed
		1 training started: {'n_factors': 20, 'n_epochs': 20, 'reg_all': 0.02}
		RMSE=  1.104169452163266
		2 training started: {'n_factors': 10, 'n_epochs': 10, 'reg_all': 0.02}
		RMSE=  1.104018660021849
		3 training started: {'n_factors': 10, 'n_epochs': 10}
		RMSE=  1.5191963001814524
	Preparing data for Blender Completed
		Generating predictions Complete !
		Organizing data for Blender Complete !
		Determination for Blender Algorithm Complete !
		Blender Training Complete !
	SGD Performance:  1.163313891590333
		Generating predictions Complete !
		Organizing data for Blender Complete !
		Determination for Blender Algorithm Complete !
		Blender Training Complete !
	Ridge Performance:  1.1741878518117768
2 Cross Validation started
	Data Perparation Completed
		1 training started: {'n_factors': 20, 'n_epochs': 20, 'reg_all': 0.02}
		RMSE=  1.1041579396997705
		2 training started: {'n_factors': 10, 'n_epochs': 10, 'reg_all': 0.02}
		RMS

In [17]:
# Configuration 3
algorithms = [SVD, NMF, CoClustering]
hparams = [
    {
        "n_factors": 20,
        "n_epochs": 20,
        "reg_all": 0.02
    },
    {
        "n_factors": 10,
        "n_epochs": 10
    },
    {
        "n_cltr_u": 4,
        "n_cltr_i": 4,
        "n_epochs": 30,
    }
]
individual_performances3, sgd_performances3, ridge_performances3 = CV(3, algorithms, hparams)

1 Cross Validation started
	Data Perparation Completed
		1 training started: {'n_factors': 20, 'n_epochs': 20, 'reg_all': 0.02}
		RMSE=  1.1040136718387685
		2 training started: {'n_factors': 10, 'n_epochs': 10}
		RMSE=  1.5182970493570633
		3 training started: {'n_cltr_u': 4, 'n_cltr_i': 4, 'n_epochs': 30}
		RMSE=  1.2169442432729836
	Preparing data for Blender Completed
		Generating predictions Complete !
		Organizing data for Blender Complete !
		Determination for Blender Algorithm Complete !
		Blender Training Complete !
	SGD Performance:  1.1919274541577873
		Generating predictions Complete !
		Organizing data for Blender Complete !
		Determination for Blender Algorithm Complete !
		Blender Training Complete !
	Ridge Performance:  1.1741878518117768
2 Cross Validation started
	Data Perparation Completed
		1 training started: {'n_factors': 20, 'n_epochs': 20, 'reg_all': 0.02}
		RMSE=  1.1040702709711407
		2 training started: {'n_factors': 10, 'n_epochs': 10}
		RMSE=  1.517973160854

In [18]:
individual_performances1

[[1.104058928221667, 1.1163021178376735, 1.1024902835441712],
 [1.1041334500931705, 1.1163754128751944, 1.1025266403642986],
 [1.1040176547291456, 1.1163519102615207, 1.1023872126523622]]

In [19]:
sgd_performances1

[1.108210460939158, 1.105288739170464, 1.1033749008833493]

In [20]:
# First Experiment Result for SGD Blender
# algorithms = [SVD, SVD, SVD]
np.mean(sgd_performances1), np.std(sgd_performances1)

(1.1056247003309905, 0.0019883515264945873)

In [21]:
individual_performances2

[[1.104169452163266, 1.104018660021849, 1.5191963001814524],
 [1.1041579396997705, 1.1040242837478833, 1.5186261119582278],
 [1.1040802393895066, 1.1040675942725207, 1.520093995363411]]

In [22]:
sgd_performances2

[1.163313891590333, 1.1551716076213816, 1.1491646896736907]

In [23]:
# Second Experminet Result for SGD Blender
# algorithms = [SVD, SVDpp, NMF]
np.mean(sgd_performances2), np.std(sgd_performances2)

(1.155883396295135, 0.005798273366933818)

In [24]:
individual_performances3

[[1.1040136718387685, 1.5182970493570633, 1.2169442432729836],
 [1.1040702709711407, 1.5179731608544629, 1.2169442432729836],
 [1.104108333964276, 1.5206353514558368, 1.2169442432729836]]

In [25]:
sgd_performances3

[1.1919274541577873, 1.172098305204927, 1.1653601229616366]

In [26]:
# Third Experiment result for SGD Blender
# algorithms = [SVD, NMF, CoClustering]
np.mean(sgd_performances3), np.std(sgd_performances3)

(1.1764619607747837, 0.01127643233092059)